# Fine-tuning a GPT-4o-mini model for textual stance classification

This model uses the JSONL datasets for stance classification created in FEVER_set_creation.ipynb.

In [ ]:

import pandas as pd
import ast  # To safely evaluate the string representation of lists
import math
import os
import random
import nltk
import os
import sys
import datetime
import argparse
import json
import openai
from openai import OpenAI


from google.colab import drive, userdata
drive.mount('/content/drive')
# Adjust path as needed
BASE_DIR = '/content/drive/My Drive/SUNY_Poly_DSA598/'
DATA_DIR = os.path.join(BASE_DIR, 'datasets/FEVER/GPT_sets/')

# --- Download NLTK sentence tokenizer if needed ---
nltk.download('punkt')
nltk.download('punkt_tab')

# --- Set up OpenAI API key ---
api_key = userdata.get('openaikey')

client = OpenAI(api_key=api_key)
def create_ft_job(job_name, train_path, val_path):
    """
    Create a fine-tuning job using the OpenAI API.
    """
    # Upload the training file
    training_file = client.files.create(
        file=open(train_path, "rb"),
        purpose="fine-tune",

    )

    # Upload the validation file
    validation_file = client.files.create(
        file=open(val_path, "rb"),
        purpose="fine-tune"
    )

    # Create the fine-tuning job
    fine_tuning_job = client.fine_tuning.jobs.create(
        training_file=training_file.id,
        validation_file=validation_file.id,
        model="gpt-4o-mini-2024-07-18",
        seed=2025,
    )
    return fine_tuning_job.id

def get_ft_job_list():
    """
    Get the list of fine-tuning jobs using the OpenAI API.
    """
    fine_tuning_jobs = client.fine_tuning.jobs.list()
    return fine_tuning_jobs

def get_ft_job_status(job_id):
    """
    Get the status of a specific fine-tuning job using the OpenAI API.
    """
    fine_tuning_job = client.fine_tuning.jobs.retrieve(job_id)
    return fine_tuning_job

def get_ft_job_results(job_id):
    """
    Get the results of a specific fine-tuning job using the OpenAI API.
    """
    fine_tuning_job = client.fine_tuning.jobs.retrieve(job_id)
    return fine_tuning_job

# Call the function to create a fine-tuning job
clf_job_name = "GPT_clf_paper_dev_train"
clf_train_path = os.path.join(DATA_DIR, 'GPT_clf_paper_dev_train/prompt_v1_segmented_n200_04-19_001.jsonl')
clf_val_path = os.path.join(DATA_DIR, 'GPT_clf_paper_dev_valid/prompt_v1_segmented_n60_04-19_001.jsonl')
clf_job_id = create_ft_job(clf_job_name, clf_train_path, clf_val_path)
print(f"Fine-tuning job created with ID: {clf_job_id}")